# DIGS 20006/30006 : NLP Assignment 6

In [1]:
#coding=utf-8
import jieba
import nltk
from spacy import displacy 
import spacy
import zh_core_web_sm
import os
import sys
import re

In [2]:
#!python3 -m spacy download en_core_web_lg

In [3]:
nlp = spacy.load('en_core_web_lg')

In [4]:
def most_similar(word):
    by_similarity = sorted(word.vocab, key=lambda w: word.similarity(w), reverse=True)
    return [w.orth_ for w in by_similarity[:10]]

most_similar(nlp.vocab[u'panda'])

['Panda',
 'panda',
 'PANDA',
 'PANDAS',
 'pandas',
 'Pandas',
 'Tiger',
 'TIGER',
 'tiger',
 'Penguin']

## Q1a top 50 similarity words

In [5]:
def most_similar50(word):
    by_similarity = sorted(word.vocab, key=lambda w: word.similarity(w), reverse=True)
    return [str(w.orth_) + " : " + str(w.similarity(word)) for w in by_similarity[:50]] #w.orth_

most_similar50(nlp.vocab[u'panda'])

['Panda : 1.0',
 'panda : 1.0',
 'PANDA : 1.0',
 'PANDAS : 0.67385703',
 'pandas : 0.67385703',
 'Pandas : 0.67385703',
 'Tiger : 0.6157463',
 'TIGER : 0.6157463',
 'tiger : 0.6157463',
 'Penguin : 0.60118914',
 'penguin : 0.60118914',
 'PENGUIN : 0.60118914',
 'Lion : 0.5938121',
 'lion : 0.5938121',
 'LiOn : 0.5938121',
 'LION : 0.5938121',
 'monkey : 0.59324354',
 'MONKEY : 0.59324354',
 'Monkey : 0.59324354',
 'KOALA : 0.56075525',
 'koala : 0.56075525',
 'Koala : 0.56075525',
 'elephant : 0.5553588',
 'Elephant : 0.5553588',
 'ELEPHANT : 0.5553588',
 'Dragon : 0.55414057',
 'DRAGON : 0.55414057',
 'dragon : 0.55414057',
 'Bunny : 0.5367655',
 'BUNNY : 0.5367655',
 'bunny : 0.5367655',
 'GORILLA : 0.5238081',
 'Gorilla : 0.5238081',
 'gorilla : 0.5238081',
 'GorillA : 0.5238081',
 'Kitty : 0.51902014',
 'KITTY : 0.51902014',
 'kitty : 0.51902014',
 'Hippo : 0.515421',
 'hippo : 0.515421',
 'HIPPO : 0.515421',
 'Giraffe : 0.5128468',
 'GIRAFFE : 0.5128468',
 'giraffe : 0.5128468',
 

In [6]:
most_similar50(nlp.vocab[u'sushi'])

['sushi : 1.0',
 'SUSHI : 0.99999994',
 'Sushi : 0.99999994',
 'SASHIMI : 0.8078017',
 'Sashimi : 0.8078017',
 'sashimi : 0.8078017',
 'RAmen : 0.7049228',
 'Ramen : 0.7049228',
 'ramen : 0.7049228',
 'rAmen : 0.7049228',
 'RAMEN : 0.7049228',
 'SEAFOOD : 0.6845244',
 'Seafood : 0.6845244',
 'seafood : 0.6845244',
 'tempura : 0.67098546',
 'Tempura : 0.67098546',
 'TEMPURA : 0.67098546',
 'nigiri : 0.66326934',
 'Nigiri : 0.66326934',
 'PIZZA : 0.6317651',
 'pizza : 0.6317651',
 'Pizza : 0.6317651',
 'burger : 0.62611973',
 'BURGER : 0.62611973',
 'Burger : 0.62611973',
 'SANDWICHES : 0.6252691',
 'Sandwiches : 0.6252691',
 'sandwiches : 0.6252691',
 'NOODLE : 0.6222606',
 'Noodle : 0.6222606',
 'noodle : 0.6222606',
 'Tapas : 0.62212235',
 'tapas : 0.62212235',
 'steak : 0.6197223',
 'STEAK : 0.6197223',
 'Steak : 0.6197223',
 'Salad : 0.61868423',
 'salad : 0.61868423',
 'SALAD : 0.61868423',
 'appetizer : 0.6161339',
 'APPETIZER : 0.6161339',
 'Appetizer : 0.6161339',
 'RESTAURANT :

## Q1b Similarity scores for 5 words

In [7]:
tokens = nlp(u'panda sushi bamboo seaweed bear')

for token in tokens:
    for token2 in tokens[1:]:
        print(token.text, token2.text, token.similarity(token2))

panda sushi 0.36805877
panda bamboo 0.3250546
panda seaweed 0.20288438
panda bear 0.49143168
sushi sushi 1.0
sushi bamboo 0.3767498
sushi seaweed 0.44988915
sushi bear 0.19801743
bamboo sushi 0.3767498
bamboo bamboo 1.0
bamboo seaweed 0.45950678
bamboo bear 0.22684743
seaweed sushi 0.44988915
seaweed bamboo 0.45950678
seaweed seaweed 1.0
seaweed bear 0.118295185
bear sushi 0.19801743
bear bamboo 0.22684743
bear seaweed 0.118295185
bear bear 1.0


## Q2 gensim

In [8]:
# imports and set up logging
import gensim 
import logging
import glob, os
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [9]:
#I chose 饮水词 as my corpus
with open('ying_shui_ci.txt', 'r', encoding='utf-8') as f:
    content = f.read()
    content = content.replace('\t', '')
    content = content.replace('\n', '')
    content = content.replace('\u3000', '')
    content = re.sub(r'[^\w\s\d]',' ', content) 

In [10]:
content[:1000]

'梦江南昏鸦尽 小立恨因谁 急雪乍翻香阁絮 轻风吹到胆瓶梅 心字已成灰 菩萨蛮萧萧几叶风兼雨 离人偏识长更苦 欹枕数秋天 蟾蜍下早弦 夜寒惊被薄 泪与灯花落 无处不伤心 轻尘在玉琴 菩萨蛮催花未歇花奴鼓 酒醒已见残红舞 不忍覆余觞 临风泪数行 粉香看又别 空剩当时月 月也异当时 凄清照鬓丝 菩萨蛮春云吹散湘帘雨 絮黏蝴蝶飞还住 人在玉楼中 楼高四面风 柳烟丝一把 暝色笼鸳瓦 休近小阑干 夕阳无限山 菩萨蛮隔花才歇帘纤雨 一声弹指浑无语 梁燕自双归 长条脉脉垂 小屏山色远 妆薄铅华浅 独自立瑶阶 透寒金缕鞋 菩萨蛮晶帘一片伤心白 云鬟香雾成遥隔 无语问添衣 桐阴月已西 西风鸣络纬 不许愁人睡 只是去年秋 如何泪欲流 临江仙点滴芭蕉心欲碎 声声催忆当初 欲眠还展旧时书 鸳鸯小字 犹记手生疏 倦眼乍低缃帙乱 重看一半模糊 幽窗冷雨一灯孤 料应情尽 还道有情无 临江仙昨夜个人曾有约 严城玉漏三更 一钩新月几疏星 夜阑犹未寝 人静鼠窥灯 原是瞿唐风间阻 错教人恨无情 小阑干外寂无声 几回肠断处 风动护花铃 虞美人春情只到梨花薄 片片催零落 夕阳何事近黄昏 不道人间犹有未招魂 银笺别梦当时句 密绾同心苣 为伊判作梦中人 长向画图清夜唤真真 虞美人曲阑深处重相见 匀泪偎人颤 凄凉别后两应同 最是不胜清怨月明中 半生已分孤眠过 山枕檀痕 忆来何事最销魂 第一折技花样画罗裙 虞美人银床淅沥青梧老 粉秋蛩扫 采香行处蹙连钱 拾得翠翘何恨不能言 回廊一寸相思地 落月成孤倚 背灯和月就花阴 已是十年踪迹十年心 虞美人 秋夕信步 愁痕满地无人省 露湿琅影 闲阶小立倍荒凉 还剩旧时月色在潇湘 薄情转是多情累 曲曲柔肠碎 红笺向壁字模糊 忆共灯前呵手为伊书 鬓云松令枕函香 花径漏 依约相逢 絮语黄昏后 时节薄寒人病酒 铲地梨花 彻夜东风瘦 掩银屏 垂翠袖 何处吹箫 脉脉情微逗 肠断月明红豆蔻 月似当时 人似当时否 青衫湿悼亡近来无限伤心事 谁与话长更 从教分付 绿窗红泪 早雁初莺 当时领略 而今断送 总负多情 忽疑君到 漆灯风 痴数春星 沁园春丁巳重阳前三日 梦亡妇淡装素服 执手哽咽 语多不复能记 但临别有云  衔恨愿为天上月 年年犹得向郎圆  妇素未工诗 不知何以得此也 觉后感赋瞬息浮生 薄命如斯 低徊怎忘 记绣榻闲时 并吹戏雨 雕阑曲处 同倚斜阳 梦好难留 诗残莫续 赢得更深哭一场 遗容在 

In [11]:
filename ='ying_shui_ci.txt'
print(filename + " = " + str(len(content)) + " chars")

ying_shui_ci.txt = 8482 chars


In [12]:
from gensim.models import word2vec, Phrases
from gensim.models.phrases import Phraser

doc = [list(jieba.cut(content))]
sentence_stream = [w for w in doc]

trigram_sentences_project = []

#bigram = Phrases(sentence_stream, min_count=1, delimiter=b' ')
#trigram = Phrases(bigram[sentence_stream], min_count=1, delimiter=b' ')
#bigram = Phrases(sentence_stream, min_count=1, delimiter=b' ', threshold=2)
#trigram = Phrases(bigram[sentence_stream], min_count=1, delimiter=b' ') #, threshold=3

bigram = Phraser(Phrases(sentence_stream))
trigram = Phraser(Phrases(bigram[sentence_stream]))

for sent in sentence_stream:
    bigrams_ = bigram[sent]
    trigrams_ = trigram[bigram[sent]]
    trigram_sentences_project.append(trigrams_)

# Set values for various parameters
num_features = 200    # Word vector dimensionality                      
min_word_count = 1    # Minimum word count                        
num_workers = 20      # Number of threads to run in parallel
context = 5           # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words


model = word2vec.Word2Vec(trigram_sentences_project, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

vocab = list(model.wv.vocab.keys())
print(vocab[:10])

Building prefix dict from the default dictionary ...
2019-02-20 14:15:32,655 : DEBUG : Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/0d/k2cqq0217yl4b1cg6yyg6gh80000gn/T/jieba.cache
2019-02-20 14:15:33,504 : DEBUG : Dumping model to file cache /var/folders/0d/k2cqq0217yl4b1cg6yyg6gh80000gn/T/jieba.cache
Loading model cost 0.916 seconds.
2019-02-20 14:15:33,572 : DEBUG : Loading model cost 0.916 seconds.
Prefix dict has been built succesfully.
2019-02-20 14:15:33,574 : DEBUG : Prefix dict has been built succesfully.
2019-02-20 14:15:33,646 : INFO : collecting all words and their counts
2019-02-20 14:15:33,647 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2019-02-20 14:15:33,657 : INFO : collected 7266 word types from a corpus of 5370 words (unigram + bigrams) and 1 sentences
2019-02-20 14:15:33,658 : INFO : using 7266 counts as vocab in Phrases<0 vocab, min_count=5, threshold=10.0, max_vocab_size=40000000>
2019-02-20

2019-02-20 14:15:34,083 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-02-20 14:15:34,083 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-02-20 14:15:34,084 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-02-20 14:15:34,085 : INFO : EPOCH - 3 : training on 5337 raw words (4120 effective words) took 0.0s, 195555 effective words/s
2019-02-20 14:15:34,096 : INFO : worker thread finished; awaiting finish of 19 more threads
2019-02-20 14:15:34,100 : INFO : worker thread finished; awaiting finish of 18 more threads
2019-02-20 14:15:34,101 : INFO : worker thread finished; awaiting finish of 17 more threads
2019-02-20 14:15:34,101 : INFO : worker thread finished; awaiting finish of 16 more threads
2019-02-20 14:15:34,102 : INFO : worker thread finished; awaiting finish of 15 more threads
2019-02-20 14:15:34,103 : INFO : worker thread finished; awaiting finish of 14 more threads
2019-02-20 14:15:34,104 : INFO : worker thre

['梦江南', '昏鸦', '尽', ' ', '小立', '恨', '因', '谁', '急雪', '乍']


In [13]:
# Print the total number of items in our model's vocabulary
print(len(model.wv.vocab))

2594


In [14]:
model.wv.vocab

{'梦江南': <gensim.models.keyedvectors.Vocab at 0x13db15eb8>,
 '昏鸦': <gensim.models.keyedvectors.Vocab at 0x13db15e80>,
 '尽': <gensim.models.keyedvectors.Vocab at 0x13db15dd8>,
 ' ': <gensim.models.keyedvectors.Vocab at 0x13db15e10>,
 '小立': <gensim.models.keyedvectors.Vocab at 0x13db15e48>,
 '恨': <gensim.models.keyedvectors.Vocab at 0x13db15d68>,
 '因': <gensim.models.keyedvectors.Vocab at 0x13db15da0>,
 '谁': <gensim.models.keyedvectors.Vocab at 0x13db15d30>,
 '急雪': <gensim.models.keyedvectors.Vocab at 0x13db15cf8>,
 '乍': <gensim.models.keyedvectors.Vocab at 0x13db15c88>,
 '翻': <gensim.models.keyedvectors.Vocab at 0x13db15cc0>,
 '香阁': <gensim.models.keyedvectors.Vocab at 0x13db15c50>,
 '絮': <gensim.models.keyedvectors.Vocab at 0x13db15c18>,
 '轻风': <gensim.models.keyedvectors.Vocab at 0x13db15be0>,
 '吹': <gensim.models.keyedvectors.Vocab at 0x13db15ba8>,
 '到': <gensim.models.keyedvectors.Vocab at 0x13db15b38>,
 '胆瓶': <gensim.models.keyedvectors.Vocab at 0x13db15b70>,
 '梅': <gensim.models.ke

## Q2a Ten most similar words for 5 chosen words 

In [15]:
model.wv.most_similar (positive='人间')

2019-02-20 14:15:34,219 : INFO : precomputing L2-norms of word weight vectors


[(' ', 0.5817054510116577),
 ('还', 0.5764368772506714),
 ('是', 0.5461556315422058),
 ('人', 0.5435056090354919),
 ('谁', 0.5408062934875488),
 ('不', 0.5349652767181396),
 ('何处', 0.5295448899269104),
 ('吹', 0.5294234752655029),
 ('到', 0.514705240726471),
 ('浣溪沙', 0.5132975578308105)]

In [16]:
model.wv.most_similar (positive='梨花')

[('六曲', 0.2634173631668091),
 ('似白', 0.25090745091438293),
 ('西风', 0.2507537603378296),
 ('初收', 0.23965656757354736),
 ('姜女庙', 0.2382104992866516),
 ('独自', 0.23770615458488464),
 ('别', 0.23678642511367798),
 ('人称', 0.23516389727592468),
 ('紫玉', 0.23248203098773956),
 ('凉', 0.22782841324806213)]

In [17]:
model.wv.most_similar (positive='红豆')

[('嘹唳', 0.21959775686264038),
 ('劳人', 0.21949312090873718),
 ('梦亡妇', 0.21219368278980255),
 ('肠断', 0.20144842565059662),
 ('更无芳', 0.1994743049144745),
 ('飞', 0.19740183651447296),
 ('慵调', 0.19411048293113708),
 ('横塘', 0.19060108065605164),
 ('题', 0.1877986639738083),
 ('成遥', 0.18422283232212067)]

In [18]:
model.wv.most_similar (positive='冷雨')

[('总负', 0.23915725946426392),
 ('一床', 0.23058773577213287),
 ('清泪', 0.22380003333091736),
 ('与', 0.2236170768737793),
 ('梦江南', 0.21671754121780396),
 ('暖和', 0.21105852723121643),
 ('冰雪', 0.21055319905281067),
 ('窗纱', 0.2104257345199585),
 ('猎马', 0.21027226746082306),
 ('鹣', 0.20439641177654266)]

In [19]:
model.wv.most_similar (positive='月色')

[('风流', 0.24990388751029968),
 ('心期', 0.24297380447387695),
 ('满离', 0.23307999968528748),
 ('紫玉', 0.23014572262763977),
 ('慵', 0.2249077409505844),
 ('尽丝难', 0.22271275520324707),
 ('何事', 0.22142520546913147),
 ('偷拭', 0.21947084367275238),
 ('不见', 0.2172524333000183),
 ('西风', 0.21602760255336761)]

## Q2b Relative cosine similarity for two word pairs

In [20]:
print(model.wv.relative_cosine_similarity("月色", "江水",topn=10))
print(model.wv.relative_cosine_similarity("红豆", "相思",topn=10))
# From the gensim documentation: "For WordNet synonyms, if rcs(topn=10) is greater than 0.10 
# then wa and wb are more similar than any arbitrary word pairs."

0.01695007906891469
-0.03067865701048945


## Q2c Analogy

In [21]:
# analogies -- this example asks:
# "she" is to "sings" as "he" is to ...   (analogies are often written like this: "she:sings::he:?")
model.wv.most_similar(positive=['月色','江水'],negative=['冷雨'])

[('已成', 0.24550053477287292),
 ('渡江', 0.22980257868766785),
 ('高梧', 0.22432062029838562),
 ('雨余花外', 0.21950256824493408),
 ('曾染', 0.21848410367965698),
 ('便', 0.21367812156677246),
 ('赋此', 0.21208137273788452),
 ('见玉箫', 0.21185088157653809),
 ('一样', 0.20637980103492737),
 ('一别', 0.20633992552757263)]

In [22]:
model.wv.most_similar(positive=['黄叶','哽咽'],negative=['青苔'])

[('都', 0.27143192291259766),
 ('唇', 0.2671293616294861),
 ('网上', 0.2558945417404175),
 ('碎', 0.24720405042171478),
 ('北风', 0.24390962719917297),
 ('处', 0.2375689595937729),
 ('又', 0.23623016476631165),
 ('那能', 0.23542430996894836),
 ('守取', 0.23201100528240204),
 ('西风', 0.22959063947200775)]

## Q2d Prediction

In [23]:
model.predict_output_word(["心绪","冷月","沉思"], topn=30)

[('错', 0.0003855062),
 ('不眠', 0.00038550608),
 ('独自', 0.00038550608),
 ('枕', 0.00038550593),
 ('几叶风', 0.00038550587),
 ('戏', 0.00038550587),
 ('图', 0.00038550587),
 ('江南', 0.00038550585),
 ('荒城', 0.00038550585),
 ('载酒游', 0.00038550585),
 ('冷透', 0.00038550585),
 ('尘沙', 0.00038550585),
 ('瓦', 0.0003855058),
 ('园林', 0.0003855058),
 ('如丝', 0.00038550576),
 ('富贵花', 0.00038550576),
 ('留蝶', 0.00038550576),
 ('马上吟', 0.00038550576),
 ('觉魂销', 0.00038550576),
 ('不能', 0.00038550576),
 ('漏', 0.00038550576),
 ('红冷', 0.00038550576),
 ('溪头', 0.00038550576),
 ('猎马', 0.00038550576),
 ('却忆', 0.00038550576),
 ('隔银湾', 0.00038550576),
 ('迷楼', 0.0003855057),
 ('频来', 0.0003855057),
 ('乌起', 0.0003855057),
 ('阴', 0.0003855057)]

## Q3 Findings

I think using gensim word2vec to train chinese classical poetry is by far the most interesting. My understand for classical poetry is they are beautiful and kind of vague. I find jieba does a good job tokenizing them since they are much shorter comparing to Chinese literature. Building the chinese poetry model in gensim is very fast and decent. As for what the model identifies as “similar” terms, I think some make senses and some do not. I think word vectors in general is a pretty cool idea and I look forward to learn more about it and its application in literature.

## Q4 TF-IDF

In [24]:
#https://blog.csdn.net/meiqi0538/article/details/80218870

import jieba
import jieba.analyse
 
keywords = jieba.analyse.extract_tags(content, topK=50, withWeight=True, allowPOS=('n','nr','ns'))
 
# print(type(keywords))
# <class 'list'>

for item in keywords:
    print(item[0],item[1])

浣溪沙 0.16735903179341463
采桑子 0.15821909522601627
西风 0.12412345068662603
萧萧 0.10263682750353659
清平乐 0.07285728255536586
东风 0.0692715570224065
菩萨 0.06782003464839838
天涯 0.06129002170731707
落花 0.0590408291803252
夕阳 0.05611497946933333
临江仙 0.04757501249837399
虞美人 0.0460801963191057
多情 0.04501389667078049
阑干 0.04295131860617886
人间 0.04219643743187805
如梦令 0.04163273288878049
碧落 0.03937859571609756
黄花 0.037910433631991876
屏山 0.03712445854373984
残阳 0.03599175384682927
重阳 0.035850704418658536
断肠 0.03487032137105691
风流 0.03452644666878049
黄叶 0.03414465128552845
无情 0.03322695083085366
斜阳 0.03282606556813008
时节 0.03282389245772358
秋雨 0.03200076306156097
伤心 0.030363197439186994
双鱼 0.02953394678707317
那堪 0.02915796951926829
小楼 0.028867632946764223
残灯 0.02883228319585366
红桥 0.028545007499024395
肠断 0.028288030631707317
木兰花 0.028288030631707317
不语 0.027592121137463416
芙蓉 0.027159679892943087
垂杨 0.02714168031634146
燕子 0.02670022178097561
鸳鸯 0.02664574760074797
画图 0.026053175188048776
零落 0.025776763760487

In [25]:
tags = jieba.analyse.extract_tags(content, topK=50,withWeight=True)
for tag in tags:
    print("tag:%s\t\t weight:%f"%(tag[0],tag[1]))

tag:浣溪沙		 weight:0.077827
tag:采桑子		 weight:0.073576
tag:西风		 weight:0.057721
tag:萧萧		 weight:0.047729
tag:清平乐		 weight:0.033881
tag:何事		 weight:0.033718
tag:东风		 weight:0.032213
tag:菩萨		 weight:0.031538
tag:天涯		 weight:0.028502
tag:何处		 weight:0.027659
tag:落花		 weight:0.027456
tag:夕阳		 weight:0.026095
tag:相思		 weight:0.024709
tag:凄凉		 weight:0.022526
tag:临江仙		 weight:0.022124
tag:虞美人		 weight:0.021429
tag:惆怅		 weight:0.021155
tag:多情		 weight:0.020933
tag:阑干		 weight:0.019974
tag:黄昏		 weight:0.019868
tag:人间		 weight:0.019623
tag:无聊		 weight:0.019469
tag:如梦令		 weight:0.019360
tag:一片		 weight:0.019046
tag:茫茫		 weight:0.018767
tag:分明		 weight:0.018423
tag:碧落		 weight:0.018312
tag:回首		 weight:0.017901
tag:当时		 weight:0.017678
tag:黄花		 weight:0.017629
tag:屏山		 weight:0.017264
tag:旧时		 weight:0.017105
tag:残阳		 weight:0.016737
tag:重阳		 weight:0.016672
tag:断肠		 weight:0.016216
tag:而今		 weight:0.016116
tag:风流		 weight:0.016056
tag:蝶恋花		 weight:0.016038
tag:黄叶		 weight:0.015878
tag:寄愁		 weight:0.